In [25]:
import pandas as pd
import numpy as np

In [26]:
IMREP_df = pd.read_csv("../summary_data/75bp/IMREP/IMREP_TRB_merged_extracted_features.csv")
TRUST4_df = pd.read_csv("../summary_data/75bp/TRUST4/TRUST4_TRB_merged_extracted_features.csv")
MIXCR_df = pd.read_csv("../summary_data/75bp/MIXCR/MIXCR_TRB_merged_extracted_features.csv")
TCR_df = pd.read_csv("../summary_data/original/TCR_Seq/TCR_merged_extracted_features.csv")

In [27]:
# Rename TCR SAMPLE names based on 1:1 matchup in python dictionary
Sample_name_matchup_RNA_TCR_dict = {
    "ESO1-sorted-T-cells_S13_L007" : "RearrangementDetails_ESO1_sorted_infusion",
    "INY1-sorted-T-cells_S14_L007" : "RearrangementDetails_INY1_sorted_infusion",
    "INY2-sorted-T-cells_S15_L007" : "RearrangementDetails_INY2_sorted_infusion",
    "TR2-PBMC_S12" : "RearrangementDetails_TR-PBMC",
    "SAR-11-14-12RNA_S1" : "SAR_11-14_PBMC",
    "MP-11-28-12RNA_S2" : "MP_11-28_PBMC",
    "CMT-baseline1C_CAGATC" : "Pt204_Baseline_TCR_seq",
    "HM-baseline1C_CGATGT" : "Pt310_baseline_TCRseq",
    "PT0310_S9" : "Pt310_on-tx_TCR_seq",
    "LEK-baseline_CGATGT" : "Pt294_baseline_TCR_seq",
    "LEK-OT110712A_CCGTCC" : "Pt294_on-tx_TCR_seq",
    "JSSBaseline-RNA_GTGAAA" : "Pt_308_baseline_TCR_seq",
    "RAS-baseline_TGACCA" : "Pt_325_baseline_TCR_seq",
    "PT0112-B_S3" : "Pt_112_baseline_TCR_seq",
    "PT0285-B_S5" : "Pt_285_baseline_TCR_seq"
}

for matchup in Sample_name_matchup_RNA_TCR_dict:
    TCR_df.loc[(TCR_df['Sample'] == Sample_name_matchup_RNA_TCR_dict[matchup]), 'Sample' ] = matchup

In [28]:
# Select rows with nReads greater than 1
IMREP_df = IMREP_df[IMREP_df['nReads'] > 1]
TRUST4_df = TRUST4_df[TRUST4_df['nReads'] > 1]
MIXCR_df = MIXCR_df[MIXCR_df['nReads'] > 1]
TCR_df = TCR_df[TCR_df['nReads'] > 1]

In [29]:
# Rename nReads and frequencies columns according to tool for proper merging
IMREP_df = IMREP_df.rename(columns={"nReads": "nReads_IMREP"})
TRUST4_df = TRUST4_df.rename(columns={"nReads": "nReads_TRUST4"})
MIXCR_df = MIXCR_df.rename(columns={"nReads": "nReads_MIXCR"})
TCR_df = TCR_df.rename(columns={"nReads": "nReads_TCR"})

In [30]:
# Drop rows on renal cell, lymph node, and small intestine samples
TCR_df = TCR_df[~TCR_df['Sample'].isin(['SRR5233637','SRR5233639','TCGA-CZ-4862','TCGA-CZ-5463','TCGA-CZ-5985'])]

Complete dataframe across all samples and tools

In [31]:
# Merge dataframes based on two key combination: Sample and CDR3. Outer join ensures no data is lost for instances that do not have overlap
merge_IMREP_TRUST4 = pd.merge(IMREP_df, TRUST4_df, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_IMREP_TRUST4_MIXCR = pd.merge(MIXCR_df, merge_IMREP_TRUST4, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_complete = pd.merge(TCR_df, merge_IMREP_TRUST4_MIXCR, how='outer', on=['Sample', 'CDR3']).fillna(0)

merge_complete

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4
0,MP-11-28-12RNA_S2,CASSVNPGGYNEQFF,6.0,0.0,0.0,0.0
1,MP-11-28-12RNA_S2,CASIRTRNEKLFF,2.0,0.0,0.0,0.0
2,MP-11-28-12RNA_S2,CASSPGAANTEAFF,2.0,0.0,0.0,0.0
3,MP-11-28-12RNA_S2,CASRNQGLNTEAFF,2.0,0.0,0.0,0.0
4,MP-11-28-12RNA_S2,CASSLTGNRAYNEQFF,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...
143110,TR2-PBMC_S12,CASSLAGLAGGELFF,0.0,0.0,0.0,3.0
143111,TR2-PBMC_S12,CASSTGGTGPVEQYF,0.0,0.0,0.0,2.0
143112,TR2-PBMC_S12,CASSVMGGGFTDTQYF,0.0,0.0,0.0,2.0
143113,TR2-PBMC_S12,CASSLASQGWPSSYEQYF,0.0,0.0,0.0,2.0


In [32]:
# Add the tissue type
merge_complete.loc[merge_complete['Sample']=='CMT-baseline1C_CAGATC','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='ESO1-sorted-T-cells_S13_L007','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='HM-baseline1C_CGATGT','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='INY1-sorted-T-cells_S14_L007','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='INY2-sorted-T-cells_S15_L007','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='JSSBaseline-RNA_GTGAAA','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='LEK-OT110712A_CCGTCC','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='LEK-baseline_CGATGT','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='MP-11-28-12RNA_S2','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='PT0112-B_S3','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='PT0285-B_S5','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='PT0310_S9','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='RAS-baseline_TGACCA','tissue'] = 'melanoma'
merge_complete.loc[merge_complete['Sample']=='SAR-11-14-12RNA_S1','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='TR2-PBMC_S12','tissue'] = 'PBMC'

In [33]:
# Add T cell rich or poor tissue type
merge_complete.loc[merge_complete['Sample']=='CMT-baseline1C_CAGATC','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='ESO1-sorted-T-cells_S13_L007','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='HM-baseline1C_CGATGT','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='INY1-sorted-T-cells_S14_L007','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='INY2-sorted-T-cells_S15_L007','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='JSSBaseline-RNA_GTGAAA','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='LEK-OT110712A_CCGTCC','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='LEK-baseline_CGATGT','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='MP-11-28-12RNA_S2','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='PT0112-B_S3','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='PT0285-B_S5','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='PT0310_S9','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='RAS-baseline_TGACCA','tissue_type'] = 'T_cell_poor'
merge_complete.loc[merge_complete['Sample']=='SAR-11-14-12RNA_S1','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='TR2-PBMC_S12','tissue_type'] = 'T_cell_rich'

In [34]:
# Calculate total number of reads in each sample
total_reads = merge_complete[['Sample','nReads_TCR','nReads_MIXCR','nReads_IMREP','nReads_TRUST4']].groupby('Sample').sum().rename(columns={'nReads_TCR':'total_reads_TCR','nReads_MIXCR':'total_reads_MIXCR','nReads_IMREP':'total_reads_IMREP','nReads_TRUST4':'total_reads_TRUST4'})
total_reads

,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4
Sample,,,,
CMT-baseline1C_CAGATC,906121.0,50.0,58.0,2.0
ESO1-sorted-T-cells_S13_L007,90577.0,147531.0,101887.0,41094.0
HM-baseline1C_CGATGT,1257571.0,0.0,0.0,0.0
INY1-sorted-T-cells_S14_L007,87762.0,45348.0,47713.0,31027.0
INY2-sorted-T-cells_S15_L007,305953.0,52873.0,37415.0,13885.0
JSSBaseline-RNA_GTGAAA,1408590.0,68.0,35.0,11.0
LEK-OT110712A_CCGTCC,1157845.0,209.0,46.0,8.0
LEK-baseline_CGATGT,1769522.0,24.0,4.0,3.0
MP-11-28-12RNA_S2,18617.0,48.0,88.0,0.0


In [35]:
# Merge dataframes 
merge = pd.merge(merge_complete, total_reads, how='outer', on=['Sample']).fillna(0)

# Calculate frequency of CDR3 reads with respect to CDR3s that occur more than once
merge['frequency_TCR'] = merge['nReads_TCR'] / (merge['total_reads_TCR'] * 1.0)
merge['frequency_MIXCR'] = merge['nReads_MIXCR'] / (merge['total_reads_MIXCR'] * 1.0)
merge['frequency_IMREP'] = merge['nReads_IMREP'] / (merge['total_reads_IMREP'] * 1.0)
merge['frequency_TRUST4'] = merge['nReads_TRUST4'] / (merge['total_reads_TRUST4'] * 1.0)  
merge.fillna(0, inplace=True)
merge

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4
0,MP-11-28-12RNA_S2,CASSVNPGGYNEQFF,6.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000322,0.0,0.0,0.000000
1,MP-11-28-12RNA_S2,CASIRTRNEKLFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000
2,MP-11-28-12RNA_S2,CASSPGAANTEAFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000
3,MP-11-28-12RNA_S2,CASRNQGLNTEAFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000
4,MP-11-28-12RNA_S2,CASSLTGNRAYNEQFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143110,SAR-11-14-12RNA_S1,CASSSTGGSVERNIQYF,0.0,0.0,0.0,2.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.045455
143111,SAR-11-14-12RNA_S1,CASSLEGLANGYTF,0.0,0.0,0.0,2.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.045455
143112,SAR-11-14-12RNA_S1,CASSPHPDSNQPQHF,0.0,0.0,0.0,4.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.090909
143113,SAR-11-14-12RNA_S1,CASSLGGGGANEQFF,0.0,0.0,0.0,2.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.045455


Differentiate monoclonal and polyclonal repertoire

In [36]:
# Calculate Shannon-Wiener index
diversity_TCR = merge[['Sample','tissue','tissue_type','CDR3','nReads_TCR','total_reads_TCR','frequency_TCR']]
diversity_TCR = diversity_TCR[diversity_TCR.nReads_TCR != 0]
clonotype_count_TCR = diversity_TCR.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_TCR')

diversity_TCR['shannon_index_TCR'] = -(diversity_TCR['frequency_TCR']*np.log(diversity_TCR['frequency_TCR']))
shannon_TCR = diversity_TCR.groupby(['Sample']).agg({'shannon_index_TCR':'sum'}).reset_index().rename(columns={'':"shannon_index_TCR"})
shannon_TCR = pd.merge(shannon_TCR, clonotype_count_TCR, on=['Sample'])

# Define monoclonal sample as the shannon_index < 1.5, polyclonal sample as the shannon_index >= 1.5
shannon_TCR['repertoire_type'] = ['monoclonal' if x < 1.5 else 'polyclonal' for x  in shannon_TCR['shannon_index_TCR']]
repertoire_type = shannon_TCR[['Sample','repertoire_type']]
shannon_TCR

,Sample,shannon_index_TCR,clonotype_count_TCR,repertoire_type
0,CMT-baseline1C_CAGATC,8.172351,9354,polyclonal
1,ESO1-sorted-T-cells_S13_L007,1.072675,3001,monoclonal
2,HM-baseline1C_CGATGT,4.680209,3548,polyclonal
3,INY1-sorted-T-cells_S14_L007,1.001202,3229,monoclonal
4,INY2-sorted-T-cells_S15_L007,0.960516,7697,monoclonal
5,JSSBaseline-RNA_GTGAAA,8.035551,12844,polyclonal
6,LEK-OT110712A_CCGTCC,7.651713,19018,polyclonal
7,LEK-baseline_CGATGT,7.207395,16417,polyclonal
8,MP-11-28-12RNA_S2,8.005115,5440,polyclonal
9,PT0112-B_S3,7.730309,6178,polyclonal


In [37]:
# Generate metadata 
metadata = pd.merge(merge, repertoire_type, how='outer', on=['Sample'])
metadata.loc[:,'class'] = metadata["tissue_type"] +"_"+ metadata["repertoire_type"]

metadata.to_csv('../summary_data/75bp/all_tools_TRB.csv', index=False)
metadata

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4,repertoire_type,class
0,MP-11-28-12RNA_S2,CASSVNPGGYNEQFF,6.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000322,0.0,0.0,0.000000,polyclonal,T_cell_rich_polyclonal
1,MP-11-28-12RNA_S2,CASIRTRNEKLFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000,polyclonal,T_cell_rich_polyclonal
2,MP-11-28-12RNA_S2,CASSPGAANTEAFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000,polyclonal,T_cell_rich_polyclonal
3,MP-11-28-12RNA_S2,CASRNQGLNTEAFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000,polyclonal,T_cell_rich_polyclonal
4,MP-11-28-12RNA_S2,CASSLTGNRAYNEQFF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,0.000107,0.0,0.0,0.000000,polyclonal,T_cell_rich_polyclonal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143110,SAR-11-14-12RNA_S1,CASSSTGGSVERNIQYF,0.0,0.0,0.0,2.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.045455,polyclonal,T_cell_rich_polyclonal
143111,SAR-11-14-12RNA_S1,CASSLEGLANGYTF,0.0,0.0,0.0,2.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.045455,polyclonal,T_cell_rich_polyclonal
143112,SAR-11-14-12RNA_S1,CASSPHPDSNQPQHF,0.0,0.0,0.0,4.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.090909,polyclonal,T_cell_rich_polyclonal
143113,SAR-11-14-12RNA_S1,CASSLGGGGANEQFF,0.0,0.0,0.0,2.0,PBMC,T_cell_rich,104779.0,277.0,1135.0,44.0,0.000000,0.0,0.0,0.045455,polyclonal,T_cell_rich_polyclonal


Calculate Shannon-Wiener index and absolute error for each sample across different tools

In [38]:
# MIXCR
diversity_MIXCR = merge[['Sample','tissue','tissue_type','CDR3','nReads_MIXCR','total_reads_MIXCR','frequency_MIXCR']]
diversity_MIXCR = diversity_MIXCR[diversity_MIXCR.nReads_MIXCR != 0]
clonotype_count_MIXCR = diversity_MIXCR.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_MIXCR['shannon_index_tool'] = -(diversity_MIXCR['frequency_MIXCR']*np.log(diversity_MIXCR['frequency_MIXCR']))
shannon_MIXCR = diversity_MIXCR.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_MIXCR = pd.merge(shannon_MIXCR, clonotype_count_MIXCR, on=['Sample'])
shannon_MIXCR['tool'] = 'MIXCR'

# IMREP
diversity_IMREP = merge[['Sample','tissue','tissue_type','CDR3','nReads_IMREP','total_reads_IMREP','frequency_IMREP']]
diversity_IMREP = diversity_IMREP[diversity_IMREP.nReads_IMREP != 0]
clonotype_count_IMREP = diversity_IMREP.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_IMREP['shannon_index_tool'] = -(diversity_IMREP['frequency_IMREP']*np.log(diversity_IMREP['frequency_IMREP']))
shannon_IMREP = diversity_IMREP.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_IMREP = pd.merge(shannon_IMREP, clonotype_count_IMREP, on=['Sample'])
shannon_IMREP['tool'] = 'IMREP'

# TRUST4
diversity_TRUST4 = merge[['Sample','tissue','tissue_type','CDR3','nReads_TRUST4','total_reads_TRUST4','frequency_TRUST4']]
diversity_TRUST4 = diversity_TRUST4[diversity_TRUST4.nReads_TRUST4 != 0]
clonotype_count_TRUST4 = diversity_TRUST4.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_TRUST4['shannon_index_tool'] = -(diversity_TRUST4['frequency_TRUST4']*np.log(diversity_TRUST4['frequency_TRUST4']))
shannon_TRUST4 = diversity_TRUST4.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_TRUST4 = pd.merge(shannon_TRUST4, clonotype_count_TRUST4, on=['Sample'])
shannon_TRUST4['tool'] = 'TRUST4'

diversity = pd.concat([shannon_MIXCR,shannon_IMREP,shannon_TRUST4])
diversity = pd.merge(diversity, shannon_TCR, how='outer', on=['Sample'])
diversity = diversity.fillna(0)
tissue_type = merge[['Sample','tissue','tissue_type']].drop_duplicates(keep='first')
diversity = pd.merge(diversity, tissue_type, how='inner', on=['Sample'])
diversity['absolute_error'] = np.abs(diversity['shannon_index_TCR'] - diversity['shannon_index_tool'])
diversity.loc[:,'class'] = diversity["tissue_type"] +"_"+ diversity["repertoire_type"]
diversity.to_csv('../summary_data/75bp/all_tools_TRB_diversity.csv', index=False)

diversity

,Sample,shannon_index_tool,clonotype_count_tool,tool,shannon_index_TCR,clonotype_count_TCR,repertoire_type,tissue,tissue_type,absolute_error,class
0,CMT-baseline1C_CAGATC,2.811526,18.0,MIXCR,8.172351,9354,polyclonal,melanoma,T_cell_poor,5.360825,T_cell_poor_polyclonal
1,CMT-baseline1C_CAGATC,3.319493,28.0,IMREP,8.172351,9354,polyclonal,melanoma,T_cell_poor,4.852858,T_cell_poor_polyclonal
2,CMT-baseline1C_CAGATC,0.000000,1.0,TRUST4,8.172351,9354,polyclonal,melanoma,T_cell_poor,8.172351,T_cell_poor_polyclonal
3,ESO1-sorted-T-cells_S13_L007,0.505884,1513.0,MIXCR,1.072675,3001,monoclonal,PBMC,T_cell_rich,0.566791,T_cell_rich_monoclonal
4,ESO1-sorted-T-cells_S13_L007,1.101218,1990.0,IMREP,1.072675,3001,monoclonal,PBMC,T_cell_rich,0.028543,T_cell_rich_monoclonal
5,ESO1-sorted-T-cells_S13_L007,1.368813,1448.0,TRUST4,1.072675,3001,monoclonal,PBMC,T_cell_rich,0.296138,T_cell_rich_monoclonal
6,INY1-sorted-T-cells_S14_L007,0.665529,1103.0,MIXCR,1.001202,3229,monoclonal,PBMC,T_cell_rich,0.335672,T_cell_rich_monoclonal
7,INY1-sorted-T-cells_S14_L007,2.230410,3482.0,IMREP,1.001202,3229,monoclonal,PBMC,T_cell_rich,1.229209,T_cell_rich_monoclonal
8,INY1-sorted-T-cells_S14_L007,1.892770,1694.0,TRUST4,1.001202,3229,monoclonal,PBMC,T_cell_rich,0.891568,T_cell_rich_monoclonal
9,INY2-sorted-T-cells_S15_L007,0.457852,712.0,MIXCR,0.960516,7697,monoclonal,PBMC,T_cell_rich,0.502664,T_cell_rich_monoclonal
